In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import psycopg2
from sqlalchemy import create_engine
import os

/home/aguerin/.local/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
x_train = pd.read_csv('x_train.csv')
y_train = pd.read_csv('y_train.csv')
x_test = pd.read_csv('x_test.csv')
x_test_add = pd.read_csv('additional_x_files_2EHFvuq/x_test_additional_file.csv')
x_train_add = pd.read_csv('additional_x_files_2EHFvuq/x_train_additional_file.csv')

In [3]:
x_train.head()

,emergency vehicle selection,intervention,alert reason category,alert reason,intervention on public roads,floor,location of the event,longitude intervention,latitude intervention,emergency vehicle,...,delta status preceding selection-selection,departed from its rescue center,longitude before departure,latitude before departure,delta position gps previous departure-departure,GPS tracks departure-presentation,GPS tracks datetime departure-presentation,OSRM response,OSRM estimated distance,OSRM estimated duration
0,5105452,13264186,3,2162,0,0,148.0,2.284796,48.879669,4511,...,2027,1,2.288053,48.884698,NaN,"2.289000,48.885113;2.288861,48.884998;2.288000...",2018-07-08 19:04:43;2018-07-08 19:05:55;2018-0...,"{""code"":""Ok"",""routes"":[{""geometry"":""}yjiH}x}L_...",952.5,105.8
1,4720915,12663715,3,2124,0,1,136.0,2.247464,48.818191,4327,...,28233,1,2.268519,48.823958,NaN,NaN,NaN,"{""code"":""Ok"",""routes"":[{""geometry"":""u|~hHy`zL_...",2238.5,243.2
2,5365374,13675521,3,2163,0,2,139.0,2.258328,48.836759,4509,...,1981,0,2.272944,48.850498,33.0,"2.272972,48.850498;2.269056,48.847443;2.262611...",2018-11-16 10:14:31;2018-11-16 10:15:43;2018-1...,"{""code"":""Ok"",""routes"":[{""geometry"":""_cdiHo}zLm...",3026.4,295.4
3,4741586,12695745,3,2162,0,0,136.0,2.393832,48.776526,5731,...,1842,1,2.388940,48.787732,NaN,"2.394278,48.782112;2.393639,48.776833",2018-01-15 00:42:46;2018-01-15 00:43:58,"{""code"":""Ok"",""routes"":[{""geometry"":""sywhH_pqMy...",1934.2,166.6
4,5381209,13698743,3,2162,0,3,104.0,2.458610,48.875751,5623,...,2716,1,2.439331,48.863814,NaN,"2.439472,48.863724;2.439583,48.863777;2.440917...",2018-11-24 00:37:10;2018-11-24 00:38:22;2018-1...,"{""code"":""Ok"",""routes"":[{""geometry"":""wvfiHgn{Mj...",2706.7,262.6


In [4]:
# Establishing the connection with pgsql database

conn = psycopg2.connect(os.environ['DB_POMPIERS_URL'])
cur = conn.cursor()

In [ ]:
# Exporting the data into the local database

cur.execute('CREATE SCHEMA IF NOT EXISTS data')
conn.commit()

In [5]:
def to_table_key(s): return (s.replace(' ','_').replace('-','_')).lower()
def to_exportable_df(df):
    old_cols = df.columns.tolist()
    new_cols = map(lambda x: to_table_key(x), old_cols)
    d_chcols = {k:v for (k,v) in zip(old_cols, new_cols)}
    return d_chcols

def export_df(df, name, schema="data"):
    df = df.rename(columns=to_exportable_df(df))
    engine = create_engine(os.environ['DB_POMPIERS_URL'])
    df.to_sql(name, engine, schema, if_exists='replace', index=False)

In [6]:
# Tests if the features names written in features.csv
# match with the ones provided in the training files

features = pd.read_csv('features.csv', sep=';')
l1 = x_train.columns.tolist()+y_train.columns.tolist()+x_train_add.columns.tolist()
l2 = features['feature_name'].tolist()
test = [(feat1==feat2, feat1, feat2) for (feat1, feat2) in zip(l1, l2) if feat1 != feat2]
print(len(test)==0)

False


# Starter code provided by the challenge organizer

In [ ]:
# Data reading

x_train_transit = x_train[['OSRM estimated distance','intervention on public roads','floor']]
y_train_transit = y_train[['delta departure-presentation']]
x_test_transit = x_test[['OSRM estimated distance','intervention on public roads','floor']]

# Create a predictive model for the 'delta departure-presentation'
# based on 'OSRM estimated distance', 'intervention on public roads' and 'floor'
polynomial_features= PolynomialFeatures(degree=3)
x_train_transit_poly = polynomial_features.fit_transform(x_train_transit)
model = LinearRegression()
model.fit(x_train_transit_poly, y_train_transit)

# Prediction of the 'delta selection-presentation'
x_test_transit_poly = polynomial_features.fit_transform(x_test_transit)
y_selection_presentation_predicted = y_train['delta selection-departure'].median() + model.predict(x_test_transit_poly)

# Create a submission file
submission = pd.concat([pd.DataFrame(x_test[['emergency vehicle selection']].values), 
           pd.DataFrame(np.full((len(x_test), 1), y_train['delta selection-departure'].median())), 
           pd.DataFrame(model.predict(x_test_transit_poly)), 
           pd.DataFrame(y_selection_presentation_predicted)], 
           axis=1)

submission.columns = list(y_train.columns.values)

submission.set_index('emergency vehicle selection', inplace=True)

submission.to_csv('./submission.csv', sep=",")

In [ ]:
f, ax = plt.subplots(figsize=(10,10))
plt.hist(y_selection_presentation_predicted, bins=50)
plt.xlabel('Temps en secondes')
plt.ylabel('Nombre ')
plt.plot()

In [228]:
import json
import requests
import polyline, geojson
from pprint import pprint
from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, Polyline
from geojson import LineString
from ipywidgets import Layout
defaultLayout=Layout(width='960px', height='800px')

# Takes an address located in France (str)
# Returns its coordinates ['lon', 'lat']
def geocode(address, lon=2.352222, lat=48.856613, tp="housenumber"):
    add = address.replace(' ','+').lower()
    url = "https://api-adresse.data.gouv.fr/search/?q={ADD}&lat={LAT}&lon={LON}&type={TP}&limit=1".format(
        ADD=add, LAT=lat, LON=lon, TP=tp)
    response = requests.get(url)
    return json.loads(response.text)['features'][0]['geometry']['coordinates']

# Takes two lists of coordinates of points located in Paris area ([float, float], [float, float])
# Returns the Polyline route of the fastest track calculated by ORSM (Polyline)
def route(lonlat1, lonlat2=[2.352222, 48.856613], how='car'):
    base_url = "http://0.0.0.0:5000/route/v1/{HOW}/".format(HOW=how)
    url = base_url+str(lonlat1[0])+","+str(lonlat1[1])+";"+str(lonlat2[0])+","+str(lonlat2[1])
    response = requests.get(url)
    return json.loads(response.text)['routes'][0]['geometry']

# Takes two locations located in Paris area (str, str)
# Returns a map centered on the departure address with the plot of the fastest route
def gps_track(departure, destination):
    lonlat1=geocode(departure)
    lonlat2=geocode(destination)
    track = polyline.decode(route(lonlat1, lonlat2), geojson=True)
    line = GeoJSON(data=LineString(track), style = {'color': 'blue', 'opacity':0.6, 'weight':5, 'fillOpacity':0.1})
    m = Map(layers=(basemap_to_tiles(basemaps.Hydda.Full, "2019-11-19"),),
            layout=defaultLayout,
            center=(lonlat1[1], lonlat1[0]),
            zoom=14)
    m.add_layer(line)
    return m

In [226]:
# Takes an address located in Quebec (str)
def geocode_qc(address):
    add = address.replace(' ','+').lower()
    url = "https://geocoder.ca/?locate={ADD}&json=1".format(ADD=add)
    response = requests.get(url)
    return [json.loads(response.text)["longt"],json.loads(response.text)["latt"]]

In [227]:
gps_track("2231 rue Schulz J7Y5B8", "Aéroport Trudeau H4Y1H1")

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …